In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

DATASET_DIR = r"D:/Practicle/DL/dataset/Object Detection(Ass6)/caltech-101-img"
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("Detected classes:", NUM_CLASSES)

# ============================================================
# Load MobileNetV2 (FAST)
# ============================================================

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(160, 160, 3)
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

# Custom classifier
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print(model.summary())

# ============================================================
# Train classifier layers (FAST)
# ============================================================

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=8,
    verbose=1
)

# ============================================================
# Fine-tuning (Optional)
# ============================================================

# unfreeze last 20 layers of base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    verbose=1
)

print("Training finished!")


ERROR! Session/line number was not unique in database. History logging moved to new session 70
Found 8279 images belonging to 102 classes.
Found 866 images belonging to 102 classes.
Detected classes: 102
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160 (Functional)    │ (None, 5, 5, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 102)                 │          26,214 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,612,134 (9.96 MB)

 Trainable params: 354,150 (1.35 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

None
Epoch 1/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 78s 257ms/step - accuracy: 0.3410 - loss: 3.2453 - val_accuracy: 0.6189 - val_loss: 2.1146
Epoch 2/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 61s 234ms/step - accuracy: 0.6602 - loss: 1.6844 - val_accuracy: 0.7887 - val_loss: 1.1048
Epoch 3/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 61s 235ms/step - accuracy: 0.7937 - loss: 0.9727 - val_accuracy: 0.8545 - val_loss: 0.7308
Epoch 4/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 60s 232ms/step - accuracy: 0.8482 - loss: 0.6831 - val_accuracy: 0.8776 - val_loss: 0.5599
Epoch 5/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 61s 236ms/step - accuracy: 0.8737 - loss: 0.5318 - val_accuracy: 0.8915 - val_loss: 0.4736
Epoch 6/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 61s 235ms/step - accuracy: 0.8958 - loss: 0.4287 - val_accuracy: 0.8961 - val_loss: 0.4185
Epoch 7/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 63s 242ms/step - accuracy: 0.9152 - loss: 0.3529 - val_accuracy: 0.8938 - val_loss: 0.3929
Epoch 8/8
259/259 ━━━━━━━━━━━━━━━━━━━━ 62s 239ms/step - accuracy: 0.9289 - loss: 0.29

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# ============================================================
# TESTING FUNCTION
# ============================================================

def predict_image(img_path):
    # Load image
    img = image.load_img(img_path, target_size=IMG_SIZE)

    # Convert to array
    img_array = image.img_to_array(img)

    # Normalize + expand dimensions (batch of 1)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediction
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    confidence = np.max(prediction)

    # Map index → class name
    class_labels = list(train_gen.class_indices.keys())
    class_name = class_labels[class_index]

    # Print
    print("\n===== PREDICTION RESULT =====")
    print("Image:", img_path)
    print("Predicted Class:", class_name)
    print("Confidence:", confidence)

    # Show image
    plt.imshow(image.load_img(img_path))
    plt.title(f"Prediction: {class_name}")
    plt.axis("off")
    plt.show()


# ============================================================
# EXAMPLE TEST RUN
# ============================================================

test_image_path = r"D:/Practicle/DL/dataset/Object Detection(Ass6)/caltech-101-img/accordion/image_0002.jpg"
predict_image(test_image_path)
